In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

In [ ]:
import addict
import copy
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm
from typing import List, Dict, Tuple

import celeri

plt.rcParams["text.usetex"] = (
    False  # Plotting the global model is much much faster with tex fonts turned off
)

# Read in data files, create storage dictionaries, and do basic processing

In [ ]:
# Western North America example
config_file_name = "../data/config/western_north_america_config.json"

In [ ]:
model = celeri.build_model(config_file_name)

# Plot input data

In [ ]:
celeri.plot_input_summary(model)

# Sketching out the assembly of the block model system

$$
\begin{bmatrix}
    \mathrm{geodetic \; velocities} \\
    \mathrm{plate \; rotation \; constraints} \\
    \mathrm{slip \; rate \; constraints} \\
    \mathrm{TDE \; smoothing \; pseudodata = 0} \\
    \mathrm{TDE \; rate \; constraints} \\
    \mathrm{InSAR \; LOS \; changes} 
\end{bmatrix}
=
\begin{bmatrix}
    \mathrm{(rotations-elastic \; segments) \; to \; velocities} & \mathrm{TDEs \; to \; velocities} & \mathrm{block \; strain \; rate \; to \; velocities} & \mathrm{Mogi \; to \; velocities}\\
    \mathrm{identities}                                          & 0                                   & 0 \\
    \mathrm{plate \; rotations \; to \; slip \; rates}           & 0                                   & 0 \\
    0                                                            & \mathrm{TDE \; smoothing \; matrix} & 0 \\
    0                                                            & \mathrm{identities}                 & 0 \\
    \mathrm{(rotations-elastic \; segments) \; to \; LOS}        & \mathrm{TDEs \; to \; LOS}          & \mathrm{block \; strain \; rate \; to \; velocities}
\end{bmatrix}
\begin{bmatrix}
    \mathrm{plate \; rotation \; rates} \\
    \mathrm{TDE \; slip \; rates} \\
    \mathrm{block \; strain \; rates} \\
    \mathrm{Mogi \; rates}
\end{bmatrix}
$$

# Estimate block model parameters (dense)

In [ ]:
operators, estimation = celeri.assemble_and_solve_dense(model, tde=True, eigen=False)

# Plot model summary

In [ ]:
celeri.plot_estimation_summary(model, estimation)

# Experiment: constrainted least squares
- The idea is to constrain TDE slip rates in some region

In [ ]:
index = operators.index

lower_bound = np.zeros_like(estimation.state_vector)
upper_bound = np.zeros_like(estimation.state_vector)
lower_bound[:] = -np.inf
upper_bound[:] = np.inf

# Strike-slip
lower_bound[index.tde.start_tde_col[0] : index.tde.end_tde_col[0] : 2] = -5
upper_bound[index.tde.start_tde_col[0] : index.tde.end_tde_col[0] : 2] = 5

# Dip-slip
lower_bound[index.tde.start_tde_col[0] + 1 : index.tde.end_tde_col[0] : 2] = 0
upper_bound[index.tde.start_tde_col[0] + 1 : index.tde.end_tde_col[0] : 2] = 30

In [ ]:
from scipy.optimize import lsq_linear

# Non-linear solver (unbounded)
# res = lsq_linear(
#     estimation.operator * np.sqrt(estimation.weighting_vector[:, None]),
#     estimation.data_vector * np.sqrt(estimation.weighting_vector),
#     verbose=1,
# )

# Non-linear solver (bounded [-inf, inf])
res = lsq_linear(
    estimation.operator * np.sqrt(estimation.weighting_vector[:, None]),
    estimation.data_vector * np.sqrt(estimation.weighting_vector),
    bounds=(lower_bound, upper_bound),
    verbose=1,
)

In [ ]:
import dataclasses
estimation_bounded = dataclasses.replace(estimation, state_vector=res.x)

In [ ]:
plt.figure(figsize=(15, 5))
plt.title("all")
plt.plot(estimation.state_vector, "xr", label="linear")
plt.plot(res.x, ".b", label="bounded")
plt.legend()
plt.show()

plt.figure(figsize=(15, 5))
plt.title("rotation vector estimates")
plt.plot(estimation.state_vector[0 : 3 * index.n_blocks], "xr", label="linear")
plt.plot(res.x[0 : 3 * index.n_blocks], ".b", label="bounded")
plt.legend()
plt.show()

In [ ]:
celeri.plot_estimation_summary(model, estimation_bounded)

In [ ]:
plt.plot(estimation.state_vector)
plt.plot(estimation_bounded.state_vector)

# `lsq_linear` but with LSMR solve
- Try with Ben's H-matrix preconditioner

In [ ]:
operators_block_only = celeri.build_operators(model, eigen=False, tde=False)
weighting_vector_block_only = estimation.weighting_vector[
    0 : operators_block_only.full_dense_operator.shape[0]
][:, None]
col_norms = np.linalg.norm(
    operators_block_only.full_dense_operator * np.sqrt(weighting_vector_block_only), axis=0
)

In [ ]:
# Non-linear solver (bounded [-inf, inf])
res2 = lsq_linear(
    estimation.operator * np.sqrt(estimation.weighting_vector[:, None]),
    estimation.data_vector * np.sqrt(estimation.weighting_vector),
    bounds=(lower_bound, upper_bound),
    lsq_solver="lsmr",
    verbose=1,
)

In [ ]:
print(col_norms)

In [ ]:
def matrix_summary_plot(matrix, title_string):
    small_num = np.finfo(float).eps
    matrix[matrix == 0] = small_num
    n_bins = 100
    plt.figure(figsize=(10, 10))

    plt.subplot(2, 2, 1)
    plt.title("matrix")
    plt.imshow(np.log10(np.abs(matrix)))
    plt.colorbar()

    plt.subplot(2, 2, 2)
    plt.title("matrix element frequency distribution")
    plt.hist(np.log10(np.abs(matrix.flatten())), n_bins)
    plt.yscale("log", nonpositive="clip")
    plt.xlabel("np.log10(np.abs(matrix element))")
    plt.ylabel("N")

    plt.suptitle(title_string)
    plt.show()


matrix_summary_plot(estimation.operator, "raw")
matrix_summary_plot(
    estimation.operator * np.sqrt(estimation.weighting_vector[:, None]),
    "preconditioned",
)